In [158]:
import pandas as pd

- URL : https://stackoverflow.com/questions/3368969/find-string-between-two-substrings  

Getting some sort of modification date in a cross-platform way is easy - just call os.path.getmtime(path) and you'll get the Unix timestamp of when the file at path was last modified.

Getting file creation dates, on the other hand, is fiddly and platform-dependent, differing even between the three big OSes:

On Windows, a file's ctime (documented at https://msdn.microsoft.com/en-us/library/14h5k7ff.aspx) stores its creation date. You can access this in Python through os.path.getctime() or the .st_ctime attribute of the result of a call to os.stat(). This won't work on Unix, where the ctime is the last time that the file's attributes or content were changed.
On Mac, as well as some other Unix-based OSes, you can use the .st_birthtime attribute of the result of a call to os.stat().
On Linux, this is currently impossible, at least without writing a C extension for Python. Although some file systems commonly used with Linux do store creation dates (for example, ext4 stores them in st_crtime) , the Linux kernel offers no way of accessing them; in particular, the structs it returns from stat() calls in C, as of the latest kernel version, don't contain any creation date fields. You can also see that the identifier st_crtime doesn't currently feature anywhere in the Python source. At least if you're on ext4, the data is attached to the inodes in the file system, but there's no convenient way of accessing it.

The next-best thing on Linux is to access the file's mtime, through either os.path.getmtime() or the .st_mtime attribute of an os.stat() result. This will give you the last time the file's content was modified, which may be adequate for some use cases.

Putting this all together, cross-platform code should look something like this...

In [145]:
import os
import platform

def creation_date(path_to_file):
    """
    Try to get the date that a file was created, falling back to when it was
    last modified if that isn't possible.
    See http://stackoverflow.com/a/39501288/1709587 for explanation.
    """
    if platform.system() == 'Windows':
        return os.path.getctime(path_to_file)
    else:
        stat = os.stat(path_to_file)
        try:
            return stat.st_birthtime
        except AttributeError:
            # We're probably on Linux. No easy way to get creation dates here,
            # so we'll settle for when its content was last modified.
            return stat.st_mtime

In [148]:
creation_date('./data/SaveFile.txt')

1583163298.0

In [149]:
# https://www.programiz.com/python-programming/datetime/timestamp-datetime
from datetime import datetime

timestamp = 1545730073
dt_object = datetime.fromtimestamp(timestamp)

print("dt_object =", dt_object)
print("type(dt_object) =", type(dt_object))

dt_object = 2018-12-25 18:27:53
type(dt_object) = <class 'datetime.datetime'>


In [150]:
timestamp = creation_date('./data/SaveFile.txt')
dt_object = datetime.fromtimestamp(timestamp)

print("dt_object =", dt_object)
print("type(dt_object) =", type(dt_object))

dt_object = 2020-03-03 00:34:58
type(dt_object) = <class 'datetime.datetime'>


In [15]:
import os
import os.path
dirPath = './data/'
def getFileList(dirPath=dirPath):
#     print(os.listdir(dirPath))
#         print(os.path.isdir(dirPath)) # True
    if os.path.isdir(dirPath):
        fileList = os.listdir(dirPath)
        print(fileList)
        return fileList
    else :
        print(dirPath, '라는 폴더가 존재하지 않습니다.')

In [17]:
def getFile(fileIdx, fileList ,dirPath=dirPath):
    fileName = fileList[fileIdx]
    file = dirPath + fileName
    return file

In [41]:
fileList = getFileList()
file = getFile(0, fileList)
with open(file) as f:
    saveFile=f.read()
# tmpF = pd.read_excel(./data/)
saveFile

'function PreloadFiles takes nothing returns nothing\n\n\tcall Preload( "---------------------------------------" )\n\tcall Preload( "아이디: hydran1" )\n\tcall Preload( "플레이 버전: 0.31g Lazy fix1" )\n\tcall Preload( "호환되는 버전: 0.24c~" )\n\tcall Preload( "직업: 랜서" )\n\tcall Preload( "레벨: 420" )\n\tcall Preload( "현재 경험치 / 다음 레벨까지의 경험치: 8840900 / 8883000" )\n\tcall Preload( "----------영웅 아이템----------" )\n\tcall Preload( "1. 천지창조" )\n\tcall Preload( "2. 번개의 창 티베리우스" )\n\tcall Preload( "3. 타락한 불꽃의 망토" )\n\tcall Preload( "4. 악마의 뿔" )\n\tcall Preload( "5. 혼돈의 반지 라테아" )\n\tcall Preload( "----------배낭----------" )\n\tcall Preload( "1. 다크 매터" )\n\tcall Preload( "2. 봉인된 대천사의 날개" )\n\tcall Preload( "3. 천사의 깃털 결정" )\n\tcall Preload( "4. 탄식의 갑주 레퀴엠" )\n\tcall Preload( "5. 정화의 증표" )\n\tcall Preload( "6. 순수의 강철" )\n\tcall Preload( "7. 아다만티움" )\n\tcall Preload( "8. 신의 금속" )\n\tcall Preload( "9. 악몽의 기운" )\n\tcall Preload( "10. 타락의 포션" )\n\tcall Preload( "11. 슬레이어 부츠" )\n\tcall Preload( "12. 순수의 강철" )\n\tcall

## 제대로된것

In [85]:
import re
finalList = re.findall('"(.*)"', saveFile)

## Series별로 나누기
# 컬럼1. ---------------------------------------
# 2.----------영웅 아이템----------
# 3.----------배낭----------
# 4.로드 코드 1
LoadCode = finalList[-1]
finalList = finalList[:-1]
finalList[:10]

['---------------------------------------',
 '아이디: hydran1',
 '플레이 버전: 0.31g Lazy fix1',
 '호환되는 버전: 0.24c~',
 '직업: 랜서',
 '레벨: 420',
 '현재 경험치 / 다음 레벨까지의 경험치: 8840900 / 8883000',
 '----------영웅 아이템----------',
 '1. 천지창조',
 '2. 번개의 창 티베리우스']

In [ ]:
targetList = ['---------------------------------------', '----------영웅 아이템----------', '----------배낭----------']
def setColumn(targetList, baseList):
    print(targetList)
    return [ baseList.index(target) for target in targetList ]

In [110]:
cIdx = setColumn(targetList, finalList)

['---------------------------------------', '----------영웅 아이템----------', '----------배낭----------']


In [112]:
#3 List내포문
lastIdx = len(cIdx) - 1
colPerList = [ finalList[ cIdx[idx] + 1: cIdx[idx + 1] ] if idx != ( lastIdx ) else finalList[ cIdx[idx] + 1 :] for idx in range(len(cIdx)) ]

In [115]:
myColumns = [ 'info', 'equip', 'items', 'load' ]
data = {}
for i in range(len(myColumns)):
    data[myColumns[i]] = colPerList[i] if i < ( len(myColumns) -1 ) else [LoadCode]
data

{'info': ['아이디: hydran1',
  '플레이 버전: 0.31g Lazy fix1',
  '호환되는 버전: 0.24c~',
  '직업: 랜서',
  '레벨: 420',
  '현재 경험치 / 다음 레벨까지의 경험치: 8840900 / 8883000'],
 'equip': ['1. 천지창조',
  '2. 번개의 창 티베리우스',
  '3. 타락한 불꽃의 망토',
  '4. 악마의 뿔',
  '5. 혼돈의 반지 라테아'],
 'items': ['1. 다크 매터',
  '2. 봉인된 대천사의 날개',
  '3. 천사의 깃털 결정',
  '4. 탄식의 갑주 레퀴엠',
  '5. 정화의 증표',
  '6. 순수의 강철',
  '7. 아다만티움',
  '8. 신의 금속',
  '9. 악몽의 기운',
  '10. 타락의 포션',
  '11. 슬레이어 부츠',
  '12. 순수의 강철',
  '13. 다크 크리스탈',
  '14. 플레임샤드 아머',
  '15. 백작의 낡은 망토',
  '16. 다크 매터',
  '17. 천류의 창 아글라이아',
  '18. 대지의 갑주',
  '19. 폭풍의 반지',
  '20. 바람의 조각',
  '21. 낡은 유적의 조각',
  '22. 불로의 심장',
  '23. 빛의 결정',
  '24. 다크 크리스탈',
  '----------창고----------',
  '---------------------------------------'],
 'load': ['로드 코드 1: -load PZWADswNzfUjFF6zBzNy@q1O%fxO6jso3QOMaK3hYXKu75kOFFwq$36nn01C3$FTNkad81sTa86%KKgb%sV']}

## data Dict안의 것들을 정제하기
1. : 구분
2. 숫자 삭제.

In [156]:
list(map( lambda x: x.split(':') , data['info'] ))
list(map( lambda x : x.split('.')[0] , data['info'] ))
list(map( lambda x : x.split('.')[1].strip() , data['equip'] ))
# Condition =  str이 .을 가지고 있지않다면 -1
list(map(lambda x: x.split('.')[1].strip() if x.find('.') != -1 else x , data['items'] ))
list(map( lambda x : x.split(":")[1].strip() , data['load']) )

['-load PZWADswNzfUjFF6zBzNy@q1O%fxO6jso3QOMaK3hYXKu75kOFFwq$36nn01C3$FTNkad81sTa86%KKgb%sV']

### Dict TestCase

In [119]:
data['info']

['아이디: hydran1',
 '플레이 버전: 0.31g Lazy fix1',
 '호환되는 버전: 0.24c~',
 '직업: 랜서',
 '레벨: 420',
 '현재 경험치 / 다음 레벨까지의 경험치: 8840900 / 8883000']

In [128]:
list(map( lambda x: x.split(':') , data['info'] ))

[['아이디', ' hydran1'],
 ['플레이 버전', ' 0.31g Lazy fix1'],
 ['호환되는 버전', ' 0.24c~'],
 ['직업', ' 랜서'],
 ['레벨', ' 420'],
 ['현재 경험치 / 다음 레벨까지의 경험치', ' 8840900 / 8883000']]

In [155]:
list(map( lambda x : x.split(':')[0] , data['info'] ))

['아이디', '플레이 버전', '호환되는 버전', '직업', '레벨', '현재 경험치 / 다음 레벨까지의 경험치']

In [130]:
data['equip']

['1. 천지창조', '2. 번개의 창 티베리우스', '3. 타락한 불꽃의 망토', '4. 악마의 뿔', '5. 혼돈의 반지 라테아']

In [138]:
list(map( lambda x : x.split('.')[1].strip() , data['equip'] ))

['천지창조', '번개의 창 티베리우스', '타락한 불꽃의 망토', '악마의 뿔', '혼돈의 반지 라테아']

In [139]:
data['items']

['1. 다크 매터',
 '2. 봉인된 대천사의 날개',
 '3. 천사의 깃털 결정',
 '4. 탄식의 갑주 레퀴엠',
 '5. 정화의 증표',
 '6. 순수의 강철',
 '7. 아다만티움',
 '8. 신의 금속',
 '9. 악몽의 기운',
 '10. 타락의 포션',
 '11. 슬레이어 부츠',
 '12. 순수의 강철',
 '13. 다크 크리스탈',
 '14. 플레임샤드 아머',
 '15. 백작의 낡은 망토',
 '16. 다크 매터',
 '17. 천류의 창 아글라이아',
 '18. 대지의 갑주',
 '19. 폭풍의 반지',
 '20. 바람의 조각',
 '21. 낡은 유적의 조각',
 '22. 불로의 심장',
 '23. 빛의 결정',
 '24. 다크 크리스탈',
 '----------창고----------',
 '---------------------------------------']

In [143]:
list(map(lambda x: True if x.find('.') != -1 else False , data['items'] ))

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False]

In [144]:
# Condition =  str이 .을 가지고 있지않다면 -1
list(map(lambda x: x.split('.')[1].strip() if x.find('.') != -1 else x , data['items'] ))

['다크 매터',
 '봉인된 대천사의 날개',
 '천사의 깃털 결정',
 '탄식의 갑주 레퀴엠',
 '정화의 증표',
 '순수의 강철',
 '아다만티움',
 '신의 금속',
 '악몽의 기운',
 '타락의 포션',
 '슬레이어 부츠',
 '순수의 강철',
 '다크 크리스탈',
 '플레임샤드 아머',
 '백작의 낡은 망토',
 '다크 매터',
 '천류의 창 아글라이아',
 '대지의 갑주',
 '폭풍의 반지',
 '바람의 조각',
 '낡은 유적의 조각',
 '불로의 심장',
 '빛의 결정',
 '다크 크리스탈',
 '----------창고----------',
 '---------------------------------------']

In [159]:
from collections import Counter
testitems = list(map(lambda x: x.split('.')[1].strip() if x.find('.') != -1 else x , data['items'] ))
Counter(testitems)

Counter({'다크 매터': 2,
         '봉인된 대천사의 날개': 1,
         '천사의 깃털 결정': 1,
         '탄식의 갑주 레퀴엠': 1,
         '정화의 증표': 1,
         '순수의 강철': 2,
         '아다만티움': 1,
         '신의 금속': 1,
         '악몽의 기운': 1,
         '타락의 포션': 1,
         '슬레이어 부츠': 1,
         '다크 크리스탈': 2,
         '플레임샤드 아머': 1,
         '백작의 낡은 망토': 1,
         '천류의 창 아글라이아': 1,
         '대지의 갑주': 1,
         '폭풍의 반지': 1,
         '바람의 조각': 1,
         '낡은 유적의 조각': 1,
         '불로의 심장': 1,
         '빛의 결정': 1,
         '----------창고----------': 1,
         '---------------------------------------': 1})

In [151]:
data['load']

['로드 코드 1: -load PZWADswNzfUjFF6zBzNy@q1O%fxO6jso3QOMaK3hYXKu75kOFFwq$36nn01C3$FTNkad81sTa86%KKgb%sV']

In [153]:
list(map( lambda x : x.split(":")[1].strip() , data['load']) )

['-load PZWADswNzfUjFF6zBzNy@q1O%fxO6jso3QOMaK3hYXKu75kOFFwq$36nn01C3$FTNkad81sTa86%KKgb%sV']

In [122]:
pd.Series(data['info'])
pd.Series(data['info']).apply(lambda x : x.split(':'))

0                                [아이디,  hydran1]
1                     [플레이 버전,  0.31g Lazy fix1]
2                             [호환되는 버전,  0.24c~]
3                                      [직업,  랜서]
4                                     [레벨,  420]
5    [현재 경험치 / 다음 레벨까지의 경험치,  8840900 / 8883000]
dtype: object

## Seperate  Testcase
- 내가 원하는 바를 아래에서 계속 정리

In [97]:
#1 임시로 손으로작성
test = setColumn(targetList, finalList)
finalList[0+1:7], finalList[7+1:13], finalList[13+1:]


['---------------------------------------', '----------영웅 아이템----------', '----------배낭----------']


(['아이디: hydran1',
  '플레이 버전: 0.31g Lazy fix1',
  '호환되는 버전: 0.24c~',
  '직업: 랜서',
  '레벨: 420',
  '현재 경험치 / 다음 레벨까지의 경험치: 8840900 / 8883000'],
 ['1. 천지창조', '2. 번개의 창 티베리우스', '3. 타락한 불꽃의 망토', '4. 악마의 뿔', '5. 혼돈의 반지 라테아'],
 ['1. 다크 매터',
  '2. 봉인된 대천사의 날개',
  '3. 천사의 깃털 결정',
  '4. 탄식의 갑주 레퀴엠',
  '5. 정화의 증표',
  '6. 순수의 강철',
  '7. 아다만티움',
  '8. 신의 금속',
  '9. 악몽의 기운',
  '10. 타락의 포션',
  '11. 슬레이어 부츠',
  '12. 순수의 강철',
  '13. 다크 크리스탈',
  '14. 플레임샤드 아머',
  '15. 백작의 낡은 망토',
  '16. 다크 매터',
  '17. 천류의 창 아글라이아',
  '18. 대지의 갑주',
  '19. 폭풍의 반지',
  '20. 바람의 조각',
  '21. 낡은 유적의 조각',
  '22. 불로의 심장',
  '23. 빛의 결정',
  '24. 다크 크리스탈',
  '----------창고----------',
  '---------------------------------------'])

NameError: name 'enumulate' is not defined

### Architecture
- Condition = the index is lastindex?
- finalList[sIdx + 1: eIdx] if {Condition}  else finalList[sIdx:]

In [107]:
#2 for문
lastIdx = len(test) - 1
for idx in range(len(test)):
    #     print(idx)
    tmp = finalList[ test[idx] + 1: test[idx + 1] ] if idx != ( lastIdx ) else finalList[ test[idx] + 1 :]
    print(tmp)

['아이디: hydran1', '플레이 버전: 0.31g Lazy fix1', '호환되는 버전: 0.24c~', '직업: 랜서', '레벨: 420', '현재 경험치 / 다음 레벨까지의 경험치: 8840900 / 8883000']
['1. 천지창조', '2. 번개의 창 티베리우스', '3. 타락한 불꽃의 망토', '4. 악마의 뿔', '5. 혼돈의 반지 라테아']
['1. 다크 매터', '2. 봉인된 대천사의 날개', '3. 천사의 깃털 결정', '4. 탄식의 갑주 레퀴엠', '5. 정화의 증표', '6. 순수의 강철', '7. 아다만티움', '8. 신의 금속', '9. 악몽의 기운', '10. 타락의 포션', '11. 슬레이어 부츠', '12. 순수의 강철', '13. 다크 크리스탈', '14. 플레임샤드 아머', '15. 백작의 낡은 망토', '16. 다크 매터', '17. 천류의 창 아글라이아', '18. 대지의 갑주', '19. 폭풍의 반지', '20. 바람의 조각', '21. 낡은 유적의 조각', '22. 불로의 심장', '23. 빛의 결정', '24. 다크 크리스탈', '----------창고----------', '---------------------------------------']


In [108]:
#3 List내포문
lastIdx = len(test) - 1
[ finalList[ test[idx] + 1: test[idx + 1] ] if idx != ( lastIdx ) else finalList[ test[idx] + 1 :] for idx in range(len(test)) ]

[['아이디: hydran1',
  '플레이 버전: 0.31g Lazy fix1',
  '호환되는 버전: 0.24c~',
  '직업: 랜서',
  '레벨: 420',
  '현재 경험치 / 다음 레벨까지의 경험치: 8840900 / 8883000'],
 ['1. 천지창조', '2. 번개의 창 티베리우스', '3. 타락한 불꽃의 망토', '4. 악마의 뿔', '5. 혼돈의 반지 라테아'],
 ['1. 다크 매터',
  '2. 봉인된 대천사의 날개',
  '3. 천사의 깃털 결정',
  '4. 탄식의 갑주 레퀴엠',
  '5. 정화의 증표',
  '6. 순수의 강철',
  '7. 아다만티움',
  '8. 신의 금속',
  '9. 악몽의 기운',
  '10. 타락의 포션',
  '11. 슬레이어 부츠',
  '12. 순수의 강철',
  '13. 다크 크리스탈',
  '14. 플레임샤드 아머',
  '15. 백작의 낡은 망토',
  '16. 다크 매터',
  '17. 천류의 창 아글라이아',
  '18. 대지의 갑주',
  '19. 폭풍의 반지',
  '20. 바람의 조각',
  '21. 낡은 유적의 조각',
  '22. 불로의 심장',
  '23. 빛의 결정',
  '24. 다크 크리스탈',
  '----------창고----------',
  '---------------------------------------']]

## regex TestCase

In [ ]:
?re.match

In [54]:
import re
p = re.compile('[a-z]+')
m = p.match("python")
print(m.group())

python


In [56]:
p = re.compile(r"\w+\s+\d+[-]\d+[-]\d+")
m = p.search("park 010-1234-1234")
m.group()

'park 010-1234-1234'

In [66]:

regex = re.compile('([a-zA-Z]\"[a-zA-Z])', re.S)
myfile =  'foo"s bar'
myfile2 = regex.sub(lambda m: m.group().replace('"',"%",1), myfile)
print(myfile2)


foo%s bar


In [69]:
# You can use re.sub() for the replacement
#, and re.findall() to get all matched strings.
# https://stackoverflow.com/questions/46023527/find-all-matches-of-a-pattern-and-replace-them-in-a-text

measurement = re.compile("(\d+(?:\.\d*)?)\s*x\s*(\d+(?:\.\d*)?)\s*(cm|mm|millimeter|centimeter|millimeters|centimeters)")

text = "measuring 9 x 5 mm and previously measuring 8 x 6 mm"

values = re.findall(pattern=measurement, string=text)

sub_text = re.sub(pattern=measurement, string=text, repl='MEASUREMENT')
print(values)
print(sub_text)

[('9', '5', 'mm'), ('8', '6', 'mm')]
measuring MEASUREMENT and previously measuring MEASUREMENT


In [65]:
p = re.compile(r'\n{2,}')
m = p.findall(saveFile)
m

['\n\n', '\n\n', '\n\n\n']

In [71]:
p = re.compile(r'\n{2,}')
m = re.sub(pattern=p, string=saveFile, repl='\n')
m

'function PreloadFiles takes nothing returns nothing\n\tcall Preload( "---------------------------------------" )\n\tcall Preload( "아이디: hydran1" )\n\tcall Preload( "플레이 버전: 0.31g Lazy fix1" )\n\tcall Preload( "호환되는 버전: 0.24c~" )\n\tcall Preload( "직업: 랜서" )\n\tcall Preload( "레벨: 420" )\n\tcall Preload( "현재 경험치 / 다음 레벨까지의 경험치: 8840900 / 8883000" )\n\tcall Preload( "----------영웅 아이템----------" )\n\tcall Preload( "1. 천지창조" )\n\tcall Preload( "2. 번개의 창 티베리우스" )\n\tcall Preload( "3. 타락한 불꽃의 망토" )\n\tcall Preload( "4. 악마의 뿔" )\n\tcall Preload( "5. 혼돈의 반지 라테아" )\n\tcall Preload( "----------배낭----------" )\n\tcall Preload( "1. 다크 매터" )\n\tcall Preload( "2. 봉인된 대천사의 날개" )\n\tcall Preload( "3. 천사의 깃털 결정" )\n\tcall Preload( "4. 탄식의 갑주 레퀴엠" )\n\tcall Preload( "5. 정화의 증표" )\n\tcall Preload( "6. 순수의 강철" )\n\tcall Preload( "7. 아다만티움" )\n\tcall Preload( "8. 신의 금속" )\n\tcall Preload( "9. 악몽의 기운" )\n\tcall Preload( "10. 타락의 포션" )\n\tcall Preload( "11. 슬레이어 부츠" )\n\tcall Preload( "12. 순수의 강철" )\n\tcall P

In [72]:
m.split("\n")

['function PreloadFiles takes nothing returns nothing',
 '\tcall Preload( "---------------------------------------" )',
 '\tcall Preload( "아이디: hydran1" )',
 '\tcall Preload( "플레이 버전: 0.31g Lazy fix1" )',
 '\tcall Preload( "호환되는 버전: 0.24c~" )',
 '\tcall Preload( "직업: 랜서" )',
 '\tcall Preload( "레벨: 420" )',
 '\tcall Preload( "현재 경험치 / 다음 레벨까지의 경험치: 8840900 / 8883000" )',
 '\tcall Preload( "----------영웅 아이템----------" )',
 '\tcall Preload( "1. 천지창조" )',
 '\tcall Preload( "2. 번개의 창 티베리우스" )',
 '\tcall Preload( "3. 타락한 불꽃의 망토" )',
 '\tcall Preload( "4. 악마의 뿔" )',
 '\tcall Preload( "5. 혼돈의 반지 라테아" )',
 '\tcall Preload( "----------배낭----------" )',
 '\tcall Preload( "1. 다크 매터" )',
 '\tcall Preload( "2. 봉인된 대천사의 날개" )',
 '\tcall Preload( "3. 천사의 깃털 결정" )',
 '\tcall Preload( "4. 탄식의 갑주 레퀴엠" )',
 '\tcall Preload( "5. 정화의 증표" )',
 '\tcall Preload( "6. 순수의 강철" )',
 '\tcall Preload( "7. 아다만티움" )',
 '\tcall Preload( "8. 신의 금속" )',
 '\tcall Preload( "9. 악몽의 기운" )',
 '\tcall Preload( "10. 타락의 포션" )',

In [43]:
saveFile.replace('\n\n', '\n').split('\n')

['function PreloadFiles takes nothing returns nothing',
 '\tcall Preload( "---------------------------------------" )',
 '\tcall Preload( "아이디: hydran1" )',
 '\tcall Preload( "플레이 버전: 0.31g Lazy fix1" )',
 '\tcall Preload( "호환되는 버전: 0.24c~" )',
 '\tcall Preload( "직업: 랜서" )',
 '\tcall Preload( "레벨: 420" )',
 '\tcall Preload( "현재 경험치 / 다음 레벨까지의 경험치: 8840900 / 8883000" )',
 '\tcall Preload( "----------영웅 아이템----------" )',
 '\tcall Preload( "1. 천지창조" )',
 '\tcall Preload( "2. 번개의 창 티베리우스" )',
 '\tcall Preload( "3. 타락한 불꽃의 망토" )',
 '\tcall Preload( "4. 악마의 뿔" )',
 '\tcall Preload( "5. 혼돈의 반지 라테아" )',
 '\tcall Preload( "----------배낭----------" )',
 '\tcall Preload( "1. 다크 매터" )',
 '\tcall Preload( "2. 봉인된 대천사의 날개" )',
 '\tcall Preload( "3. 천사의 깃털 결정" )',
 '\tcall Preload( "4. 탄식의 갑주 레퀴엠" )',
 '\tcall Preload( "5. 정화의 증표" )',
 '\tcall Preload( "6. 순수의 강철" )',
 '\tcall Preload( "7. 아다만티움" )',
 '\tcall Preload( "8. 신의 금속" )',
 '\tcall Preload( "9. 악몽의 기운" )',
 '\tcall Preload( "10. 타락의 포션" )',

## 제대로된것

In [74]:
re.findall('"(.*)"', saveFile)

['---------------------------------------',
 '아이디: hydran1',
 '플레이 버전: 0.31g Lazy fix1',
 '호환되는 버전: 0.24c~',
 '직업: 랜서',
 '레벨: 420',
 '현재 경험치 / 다음 레벨까지의 경험치: 8840900 / 8883000',
 '----------영웅 아이템----------',
 '1. 천지창조',
 '2. 번개의 창 티베리우스',
 '3. 타락한 불꽃의 망토',
 '4. 악마의 뿔',
 '5. 혼돈의 반지 라테아',
 '----------배낭----------',
 '1. 다크 매터',
 '2. 봉인된 대천사의 날개',
 '3. 천사의 깃털 결정',
 '4. 탄식의 갑주 레퀴엠',
 '5. 정화의 증표',
 '6. 순수의 강철',
 '7. 아다만티움',
 '8. 신의 금속',
 '9. 악몽의 기운',
 '10. 타락의 포션',
 '11. 슬레이어 부츠',
 '12. 순수의 강철',
 '13. 다크 크리스탈',
 '14. 플레임샤드 아머',
 '15. 백작의 낡은 망토',
 '16. 다크 매터',
 '17. 천류의 창 아글라이아',
 '18. 대지의 갑주',
 '19. 폭풍의 반지',
 '20. 바람의 조각',
 '21. 낡은 유적의 조각',
 '22. 불로의 심장',
 '23. 빛의 결정',
 '24. 다크 크리스탈',
 '----------창고----------',
 '---------------------------------------',
 '로드 코드 1: -load PZWADswNzfUjFF6zBzNy@q1O%fxO6jso3QOMaK3hYXKu75kOFFwq$36nn01C3$FTNkad81sTa86%KKgb%sV']